In [1]:
from IPython.display import Image

- https://www.bilibili.com/video/BV1TMRHYJEaw/

## qwen-vl

In [2]:
Image(url='./imgs/qwen-vl-pipeline.png', width=400)

- Visual Encoder
    - pre-trained weights from Openclip’s ViT-bigG
- (Position-aware Vision-Language) Adapter
    - single-layer cross-attention module
        - a group of trainable vectors (Embeddings) as **query** vectors
        - image features from the visual encoder as **keys** for crossattention operations
        - 2D absolute positional encodings
- inputs/outputs
    - `<img></img>`
    - `<box></box>`

### Data Format of Multi-Task Pre-training

In [3]:
Image(url='./imgs/qwen-vl-pretrain.png', width=600)

### Supervised Fine-tuning

In [4]:
Image(url='./imgs/qwen-vl-sft.png', width=600)

## qwen2.5-vl

### naive dynamic resolution (原生动态分辨率)

-  (1092, 8204)
    - 在训练和推理过程中，输入图像的高度和宽度在送入ViT之前会被调整为28的倍数。
        - 宽度 1092 / 28 = 39 (已经是28的倍数)
        - 高度 8204 / 28 = 293 (已经是28的倍数)
    - 图像切块 (Patching)：
        - 视觉编码器通过将图像分割成 **步长（stride）为14的块（patches）** 来处理图像，生成一组图像特征。
            - 我们使用 14×14 的图像块作为基本单元”。
            - 这意味着图像会被切分成 14x14 大小的、不重叠的块。
        - 在宽度方向上的块数量：1092 / 14 = 78 个块；在高度方向上的块数量：8204 / 14 = 586 个块
            - 所以，ViT最初生成的原始图像特征（或称为“视觉token”）数量是：78 * 586 = 45708 个原始视觉token。
    - ViT Hidden Size
        - "Vision Transformer (ViT)" 部分的 "Hidden Size" 为 1280。
    - Vision-Language Merger
        - 我们首先将空间上相邻的四个图像块特征进行分组。这些分组后的特征随后被连接并通过一个两层的多层感知器（MLP）投影到一个与LLM中使用的文本嵌入对齐的维度。这种方法不仅降低了计算成本，还提供了一种灵活的方式来动态压缩不同长度的图像特征序列。
            - 45708/4 = 11427
        - 4 × 1280 = 5120维 => 2层 MLP
            - Qwen2.5-VL-72B模型，LLM的隐藏层维度是 8192 (根据表1 LLM Hidden Size)。

### 动态 fps 采样训练

- 考虑 8s 的视频
    - 单纯抽帧会丢失时间间隔信息
        - 3帧：0s/4s/8s
        - 5帧：0s/2s/4s/6s/8s
    - 训练过程中对视频动态采样：1/8fps => 8fps
- mRoPE：Multimodal RoPE
    - Qwen2.5-VL将 MROPE 的时间维度与视频中的绝对时间对齐。如图1所示，有"Absolute time: 0.0s 0.5s 1.0s ... 8.0s"。
        - 如果采样是 0.5 FPS，那么8秒视频会采样 8 * 0.5 = 4 帧。
            - Conv3D(2\*14\*14) => 2 帧，(0, 15)
        - 如果采样是 1 FPS，那么8秒视频会采样 8 * 1 = 8 帧。
            - Conv3D(2\*14\*14) => 4 帧，(0, 5, 10, 15)
        - 如果采样是 2 FPS，那么8秒视频会采样 8 * 2 = 16 帧。
            - Conv3D(2\*14\*14) => 8 帧，(0, 2, 4, 6, 9, 11, 13, 15)
- window attention vs. full attention
    - (112\*112) 经过 (14\*14) 的卷积得到 (8*8), 64 个 patches/tokens
        - windowed attention with a maximum window size of 112×112 (corresponding to 8×8 patches
    - 大部分层 (28层) 使用 Window Attention，（1/2/3/4/5/6, 8/9/10/11/12/13/14, ...）
        - 图像（或视频帧的特征图）被划分为多个112x112像素的窗口（每个窗口包含8x8=64个14x14的patches/tokens）。注意力计算只在这些窗口内部进行。
    - 少数特定层 (4层，如第7, 15, 23, 31层) 使用 Full Attention:

## RoPE => 2d-RoPE => mRoPE

### RoPE

- 对于一个 query 向量 $q_m$ (在位置m) 和一个 key 向量 $k_n$ (在位置n)，它们分别乘以旋转矩阵 $R_m$ 和 $R_n$。
    - 注意力得分 $(R_m\cdot q)^T (R_n\cdot k)$ 可以被证明等价于 $q^T R_{m-n}^T k$ (这里 $R_{m-n}$ 是一个只依赖于相对位置 $m-n$ 的旋转矩阵)。
- 优点
    - 能够很好地捕捉相对位置关系。
    - 具有较好的外推性，即对于训练时未见过的序列长度，仍能保持一定的性能。
    - 不需要学习额外的参数（与可学习的绝对位置编码相比）。

### 2D-RoPE

> For positional encoding, we adopt 2D Rotary Positional Embedding (RoPE) to effectively capture spatial relationships in 2D space

- 当处理图像时，每个图像块 (patch) 不再是线性序列中的一个点，而是在一个二维网格中，具有行 (height/row) 和列 (width/column) 两个维度的位置。2D-RoPE 就是将1D RoPE的思想扩展到这两个维度上。
    - 为 Vision Transformer (ViT) 中的图像块嵌入提供二维空间位置信息，使其能够理解图像块之间的空间排布关系（例如，哪些块在上方，哪些在左侧，哪些是相邻的等）
    - 实现方式
        - 维度划分：将图像块的嵌入向量的维度（例如，ViT的Hidden Size是1280）分成两部分。
            - 例如，前一半维度 (如 0 到 639) 用于编码行位置 (height/row index)。
            - 后一半维度 (如 640 到 1279) 用于编码列位置 (width/column index)。
        - 独立应用1D RoPE：
            - 对于前一半维度，根据该图像块的行索引应用1D RoPE。
            - 对于后一半维度，根据该图像块的列索引应用1D RoPE。
        - 这样，一个图像块 P(r, c) (在第 r 行，第 c 列) 的嵌入向量，其一部分会根据 r 进行旋转，另一部分会根据 c 进行旋转。
    - 当计算两个图像块 P(r1, c1) 和 P(r2, c2) 之间的注意力时，注意力得分会同时受到它们行方向的相对位置 (r1-r2) 和列方向的相对位置 (c1-c2) 的影响。

*   令 $\mathbf{q}$ 为图像块 $P(r_1, c_1)$ 的原始（无位置编码的）查询向量。
*   令 $\mathbf{k}$ 为图像块 $P(r_2, c_2)$ 的原始（无位置编码的）键向量。
*   我们将查询向量 $\mathbf{q}$ 和键向量 $\mathbf{k}$ 的维度分成两部分，分别对应行 (height, $h$) 和列 (width, $w$) 的位置编码：
    $$
    \mathbf{q} = \begin{pmatrix} \mathbf{q}_h \\ \mathbf{q}_w \end{pmatrix}, \quad
    \mathbf{k} = \begin{pmatrix} \mathbf{k}_h \\ \mathbf{k}_w \end{pmatrix}
    $$
    其中 $\mathbf{q}_h, \mathbf{k}_h$ 是用于行位置编码的部分，$\mathbf{q}_w, \mathbf{k}_w$ 是用于列位置编码的部分。
*   令 $\mathcal{R}_h(pos)$ 表示应用于行部分的、与行位置 $pos$ 相关的1D RoPE旋转操作。
*   令 $\mathcal{R}_w(pos)$ 表示应用于列部分的、与列位置 $pos$ 相关的1D RoPE旋转操作。
  
经过2D-RoPE处理后，查询向量 $\mathbf{q}'$ 和键向量 $\mathbf{k}'$ 变为：

$$
\mathbf{q}' = \begin{pmatrix} \mathcal{R}_h(r_1)\mathbf{q}_h \\ \mathcal{R}_w(c_1)\mathbf{q}_w \end{pmatrix}
$$

$$
\mathbf{k}' = \begin{pmatrix} \mathcal{R}_h(r_2)\mathbf{k}_h \\ \mathcal{R}_w(c_2)\mathbf{k}_w \end{pmatrix}
$$

**3. 计算注意力得分 (点积注意力)**:

注意力得分 $A$ 是 $\mathbf{q}'$ 和 $\mathbf{k}'$ 的点积 (忽略缩放因子 $1/\sqrt{d_k}$):

$$
A = \mathbf{q}'^T \mathbf{k}'
$$

代入 $\mathbf{q}'$ 和 $\mathbf{k}'$:

$$
A = \begin{pmatrix} \mathcal{R}_h(r_1)\mathbf{q}_h \\ \mathcal{R}_w(c_1)\mathbf{q}_w \end{pmatrix}^T \begin{pmatrix} \mathcal{R}_h(r_2)\mathbf{k}_h \\ \mathcal{R}_w(c_2)\mathbf{k}_w \end{pmatrix}
$$

$$
A = \left( (\mathcal{R}_h(r_1)\mathbf{q}_h)^T \quad (\mathcal{R}_w(c_1)\mathbf{q}_w)^T \right) \begin{pmatrix} \mathcal{R}_h(r_2)\mathbf{k}_h \\ \mathcal{R}_w(c_2)\mathbf{k}_w \end{pmatrix}
$$

展开点积：

$$
A = (\mathcal{R}_h(r_1)\mathbf{q}_h)^T (\mathcal{R}_h(r_2)\mathbf{k}_h) + (\mathcal{R}_w(c_1)\mathbf{q}_w)^T (\mathcal{R}_w(c_2)\mathbf{k}_w)
$$

1D RoPE的一个关键特性是，对于任意向量 $\mathbf{x}, \mathbf{y}$ 和位置 $m, n$:
$$ (\mathcal{R}(m)\mathbf{x})^T (\mathcal{R}(n)\mathbf{y}) = \mathbf{x}^T \mathcal{R}_{m-n}^T \mathbf{y} $$
其中 $\mathcal{R}_{m-n}$ 是一个仅依赖于相对位置 $m-n$ 的旋转操作。

我们将此特性分别应用于注意力得分 $A$ 的两个组成部分：

*   **行方向部分**:
    令 $\Delta r = r_1 - r_2$。
    $$
    (\mathcal{R}_h(r_1)\mathbf{q}_h)^T (\mathcal{R}_h(r_2)\mathbf{k}_h) = \mathbf{q}_h^T \mathcal{R}_{h, \Delta r}^T \mathbf{k}_h
    $$
    这一项仅依赖于行方向的相对位置 $\Delta r = r_1-r_2$ 以及原始嵌入的行对应部分 $\mathbf{q}_h, \mathbf{k}_h$。

*   **列方向部分**:
    令 $\Delta c = c_1 - c_2$。
    $$
    (\mathcal{R}_w(c_1)\mathbf{q}_w)^T (\mathcal{R}_w(c_2)\mathbf{k}_w) = \mathbf{q}_w^T \mathcal{R}_{w, \Delta c}^T \mathbf{k}_w
    $$
    这一项仅依赖于列方向的相对位置 $\Delta c = c_1-c_2$ 以及原始嵌入的列对应部分 $\mathbf{q}_w, \mathbf{k}_w$。


**5. 最终的注意力得分**:

将上述结果代回注意力得分 $A$ 的表达式：

$$
A = \mathbf{q}_h^T \mathcal{R}_{h, (r_1-r_2)}^T \mathbf{k}_h + \mathbf{q}_w^T \mathcal{R}_{w, (c_1-c_2)}^T \mathbf{k}_w
$$
